# 青空文庫から作家別に作品をダウンロードして前処理して保存

https://qiita.com/dzbt_dzbt/items/593dbd698a07c12a771c

@dzbt_dzbtさんの記事を用いてColabで動作するように改変しました。有益な記事ありがとうございます。前処理の内容は記事をご覧ください。


https://www.aozora.gr.jp/

青空文庫の活動にも感謝。

### import

In [ ]:
# import
import pandas as pd
from pathlib import Path
import os

## ダウンロードおよび保存形式の指定

In [ ]:
#@title 設定
#@markdown 青空文庫の作家番号(6桁)
author_id = "001779" #@param {type:"string"}
#@markdown 青空文庫の表記での作家名
author_name = '江戸川乱歩'  #@param {type:"string"}

#@markdown 2カラム目に作品名を入れるか
write_title = False #@param {type:"boolean"}
#@markdown 1行目をカラム名にするか（カラム名「text」「title」)
write_header = False #@param {type:"boolean"}
#@markdown 元データをUTF-8にしたテキストファイルを保存するか
save_utf8_org = True #@param {type:"boolean"}

# 保存先
#@markdown 保存先最後の/は不要です。
SAVE_PATH = '/content' #@param {type:"string"}
out_dir = Path(SAVE_PATH + f'/out_{author_id}/')  # ファイル出力先
tx_org_dir = Path(out_dir / './org/')  # 元テキストのUTF-8変換ファイルの保存先
tx_edit_dir = Path(out_dir / './edit/')  # テキスト整形後のファイル保存先

# ダウンロード

In [ ]:
# 念のため設定を確認してみます。
print('SAVE PATH:', out_dir)

SAVE PATH: /content/out_001779


In [ ]:
# Colabにsubversion入ってなかったのでinstall
!apt install -y subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
subversion is already the newest version (1.9.7-4ubuntu1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
# 作家別のURLを定義
author_url = r'https://github.com/aozorabunko/aozorabunko/trunk/cards/' + author_id + '/'
os.environ['aurl'] = author_url
print('Import files from :',author_url)
# svnでダウンロード
!svn export $aurl

Down load files from : https://github.com/aozorabunko/aozorabunko/trunk/cards/001779/
A    001779
A    001779/card56645.html
A    001779/card56646.html
A    001779/card56647.html
A    001779/card56648.html
A    001779/card56649.html
A    001779/card56650.html
A    001779/card56651.html
A    001779/card56658.html
A    001779/card56669.html
A    001779/card56670.html
A    001779/card56671.html
A    001779/card56672.html
A    001779/card56673.html
A    001779/card56674.html
A    001779/card56675.html
A    001779/card56676.html
A    001779/card56677.html
A    001779/card56678.html
A    001779/card56679.html
A    001779/card56680.html
A    001779/card56681.html
A    001779/card56682.html
A    001779/card56683.html
A    001779/card56684.html
A    001779/card56686.html
A    001779/card56687.html
A    001779/card56688.html
A    001779/card56689.html
A    001779/card56690.html
A    001779/card56691.html
A    001779/card57105.html
A    001779/card57106.html
A    001779/card57107.html
A    001779

# 前処理

In [ ]:
# 
import pandas as pd
from pathlib import Path

def text_cleanse_df(df):
    # 本文の先頭を探す（'---…'区切りの直後から本文が始まる前提）
    head_tx = list(df[df['text'].str.contains(
        '-------------------------------------------------------')].index)
    # 本文の末尾を探す（'底本：'の直前に本文が終わる前提）
    atx = list(df[df['text'].str.contains('底本：')].index)
    if head_tx == []:
        # もし'---…'区切りが無い場合は、作家名の直後に本文が始まる前提
        head_tx = list(df[df['text'].str.contains(author_name)].index)
        head_tx_num = head_tx[0]+1
    else:
        # 2個目の'---…'区切り直後から本文が始まる
        head_tx_num = head_tx[1]+1
    df_e = df[head_tx_num:atx[0]]

    # 青空文庫の書式削除
    df_e = df_e.replace({'text': {'《.*?》': ''}}, regex=True)
    df_e = df_e.replace({'text': {'［.*?］': ''}}, regex=True)
    df_e = df_e.replace({'text': {'｜': ''}}, regex=True)

    # 字下げ（行頭の全角スペース）を削除
    df_e = df_e.replace({'text': {'　': ''}}, regex=True)

    # 節区切りを削除
    df_e = df_e.replace({'text': {'^.$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^―――.*$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^＊＊＊.*$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^×××.*$': ''}}, regex=True)

    # 記号、および記号削除によって残ったカッコを削除
    df_e = df_e.replace({'text': {'―': ''}}, regex=True)
    df_e = df_e.replace({'text': {'…': ''}}, regex=True)
    df_e = df_e.replace({'text': {'※': ''}}, regex=True)
    df_e = df_e.replace({'text': {'「」': ''}}, regex=True)

    # 一文字以下で構成されている行を削除
    df_e['length'] = df_e['text'].map(lambda x: len(x))
    df_e = df_e[df_e['length'] > 1]

    # インデックスがずれるので振りなおす
    df_e = df_e.reset_index().drop(['index'], axis=1)

    # 空白行を削除する（念のため）
    df_e = df_e[~(df_e['text'] == '')]

    # インデックスがずれるので振り直し、文字の長さの列を削除する
    df_e = df_e.reset_index().drop(['index', 'length'], axis=1)
    return df_e


def save_cleanse_text(target_file):
    try:
        # ファイルの読み込み
        print(target_file)
        # Pandas DataFrameとして読み込む（cp932で読み込まないと異体字が読めない）
        df_tmp = pd.read_csv(target_file, encoding='cp932', names=['text'])
        # 元データをUTF-8に変換してテキストファイルを保存
        if save_utf8_org:
            out_org_file_nm = Path(target_file.stem + '_org_utf-8.tsv')
            df_tmp.to_csv(Path(tx_org_dir / out_org_file_nm), sep='\t',
                          encoding='utf-8', index=None)
        # テキスト整形
        df_tmp_e = text_cleanse_df(df_tmp)
        if write_title:
            # タイトル列を作る
            df_tmp_e['title'] = df_tmp['text'][0]
        out_edit_file_nm = Path(target_file.stem + '_clns_utf-8.txt')
        df_tmp_e.to_csv(Path(tx_edit_dir / out_edit_file_nm), sep='\t',
                        encoding='utf-8', index=None, header=write_header)
    except:
        print(f'ERROR: {target_file}')

In [ ]:
# main

def main():
    tx_dir = Path(str(author_id) + '/files/')
    print('Target dir:', tx_dir)
    # zipファイルのリストを作成
    zip_list = list(tx_dir.glob('*.zip'))
    # 保存ディレクトリを作成しておく
    tx_edit_dir.mkdir(exist_ok=True, parents=True)
    if save_utf8_org:
        tx_org_dir.mkdir(exist_ok=True, parents=True)

    for target_file in zip_list:
        save_cleanse_text(target_file)

main()

Target dir: 001779/files
001779/files/57515_ruby_69288.zip
ERROR: 001779/files/57515_ruby_69288.zip
001779/files/57240_ruby_60876.zip
001779/files/56647_ruby_58166.zip
ERROR: 001779/files/56647_ruby_58166.zip
001779/files/57190_ruby_58233.zip
001779/files/57243_ruby_71258.zip
ERROR: 001779/files/57243_ruby_71258.zip
001779/files/57193_ruby_59534.zip
001779/files/57506_ruby_61212.zip
001779/files/57504_ruby_62502.zip
001779/files/58486_ruby_68458.zip
001779/files/57533_ruby_74035.zip
ERROR: 001779/files/57533_ruby_74035.zip
001779/files/57228_ruby_58697.zip
001779/files/57194_ruby_60070.zip
001779/files/57502_ruby_62060.zip
ERROR: 001779/files/57502_ruby_62060.zip
001779/files/57241_ruby_73666.zip
ERROR: 001779/files/57241_ruby_73666.zip
001779/files/56645_ruby_58194.zip
001779/files/57521_ruby_69514.zip
001779/files/58039_ruby_61534.zip
001779/files/57184_ruby_63217.zip
001779/files/56672_ruby_61189.zip
001779/files/57225_ruby_58894.zip
001779/files/57182_ruby_60006.zip
001779/files/58

# 処理後のデータをダウンロード

zipファイルにして、ダウンロード

In [ ]:
from google.colab import files
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/aozora_files.zip $out_dir
# Colabからダウンロード
files.download('/content/aozora_files.zip')

  adding: content/out_001779/ (stored 0%)
  adding: content/out_001779/org/ (stored 0%)
  adding: content/out_001779/org/56681_ruby_63206_org_utf-8.tsv (deflated 73%)
  adding: content/out_001779/org/57505_ruby_61213_org_utf-8.tsv (deflated 62%)
  adding: content/out_001779/org/57519_ruby_73853_org_utf-8.tsv (deflated 65%)
  adding: content/out_001779/org/57520_ruby_67389_org_utf-8.tsv (deflated 62%)
  adding: content/out_001779/org/57184_ruby_63217_org_utf-8.tsv (deflated 68%)
  adding: content/out_001779/org/57509_ruby_64324_org_utf-8.tsv (deflated 59%)
  adding: content/out_001779/org/57226_ruby_58864_org_utf-8.tsv (deflated 75%)
  adding: content/out_001779/org/57518_ruby_65931_org_utf-8.tsv (deflated 63%)
  adding: content/out_001779/org/56683_ruby_63024_org_utf-8.tsv (deflated 73%)
  adding: content/out_001779/org/57511_ruby_65185_org_utf-8.tsv (deflated 66%)
  adding: content/out_001779/org/56651_ruby_58728_org_utf-8.tsv (deflated 65%)
  adding: content/out_001779/org/56669_ruby

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>